In [5]:
# here for libs
import pandas as pd

Project plan:
https://docs.google.com/document/d/1o2FlMo1je0yv7LeCV_mMHfmbbjH_XfzrfiKAefOh450/edit

In [6]:
#data import
data = pd.read_csv('./dataset/diabetes_dataset00.csv')
data.head()

,Target,Genetic Markers,Autoantibodies,Family History,Environmental Factors,Insulin Levels,Age,BMI,Physical Activity,Dietary Habits,...,Pulmonary Function,Cystic Fibrosis Diagnosis,Steroid Use History,Genetic Testing,Neurological Assessments,Liver Function Tests,Digestive Enzyme Levels,Urine Test,Birth Weight,Early Onset Symptoms
0,Steroid-Induced Diabetes,Positive,Negative,No,Present,40,44,38,High,Healthy,...,76,No,No,Positive,3,Normal,56,Ketones Present,2629,No
1,Neonatal Diabetes Mellitus (NDM),Positive,Negative,No,Present,13,1,17,High,Healthy,...,60,Yes,No,Negative,1,Normal,28,Glucose Present,1881,Yes
2,Prediabetic,Positive,Positive,Yes,Present,27,36,24,High,Unhealthy,...,80,Yes,No,Negative,1,Abnormal,55,Ketones Present,3622,Yes
3,Type 1 Diabetes,Negative,Positive,No,Present,8,7,16,Low,Unhealthy,...,89,Yes,No,Positive,2,Abnormal,60,Ketones Present,3542,No
4,Wolfram Syndrome,Negative,Negative,Yes,Present,17,10,17,High,Healthy,...,41,No,No,Positive,1,Normal,24,Protein Present,1770,No


## 1.Preprocessing


In [7]:
##check missing data
missing_values = data.isnull().sum()
missing_values[missing_values > 0]


Series([], dtype: int64)

In [16]:
##lable
from sklearn.preprocessing import LabelEncoder


le = LabelEncoder()


categorical_columns = ['Genetic Markers', 'Autoantibodies', 'Family History' , 'Environmental Factors', 'Dietary Habits', 'Ethnicity', 'Socioeconomic Factors', 'Smoking Status', 'Alcohol Consumption', 'Glucose Tolerance Test', 'Previous Gestational Diabetes', 'Pregnancy History', 'Cystic Fibrosis Diagnosis', 'Environmental Factors', 'Steroid Use History', 'Genetic Testing', 'Liver Function Tests', 'Urine Test', 'Early Onset Symptoms', 'Physical Activity', 'History of PCOS', 'Target']


for col in categorical_columns:
    data[col] = le.fit_transform(data[col])


data.head(50)



,Target,Genetic Markers,Autoantibodies,Family History,Environmental Factors,Insulin Levels,Age,BMI,Physical Activity,Dietary Habits,...,Pulmonary Function,Cystic Fibrosis Diagnosis,Steroid Use History,Genetic Testing,Neurological Assessments,Liver Function Tests,Digestive Enzyme Levels,Urine Test,Birth Weight,Early Onset Symptoms
0,7,1,0,0,1,40,44,38,0,0,...,76,0,0,1,3,1,56,1,2629,0
1,4,1,0,0,1,13,1,17,0,0,...,60,1,0,0,1,1,28,0,1881,1
2,5,1,1,1,1,27,36,24,0,1,...,80,1,0,0,1,0,55,1,3622,1
3,8,0,1,0,1,8,7,16,1,1,...,89,1,0,1,2,0,60,1,3542,0
4,12,0,0,1,1,17,10,17,0,0,...,41,0,0,1,1,1,24,3,1770,0
5,2,1,0,1,1,17,41,26,2,0,...,85,1,0,0,2,1,52,1,3835,1
6,9,0,0,0,0,29,30,31,2,0,...,64,1,1,0,3,0,96,1,4426,0
7,11,1,0,0,0,10,3,18,1,1,...,44,1,0,0,1,1,29,1,1644,1
8,6,0,1,0,0,47,47,25,0,0,...,71,0,1,1,3,1,74,1,3721,0
9,6,1,0,1,1,21,72,24,1,1,...,69,1,1,1,2,0,42,3,4206,0


## 2.Validation methods

## 3.ML algorithms

## 4.Experimental results